> ⚠️ Note: Run these commands in your **Kali VM**. Some commands disable/re-enable the network interface — be careful if you are using remote SSH.

---

# Quick Commands — Copy/Paste

## 1) Information and Initial Checks

```bash
# Show interfaces and MACs
ip link show
ip addr show

# Show only eth0 (replace if your interface is different)
ip link show eth0
ip addr show eth0

# Show routing table
ip route

# Check DNS and resolution
cat /etc/resolv.conf
nslookup tryhackme.com
```

---

## 2) Install dhclient (if missing)

```bash
# Update repo list and install DHCP client
sudo apt update
sudo apt install -y isc-dhcp-client
```

---

## 3) Spoof (Change) MAC Address Temporarily

Replace `00:xx:xx:xx:xx:xx` with the MAC you want to use (e.g., Alice's).

```bash
# Disable interface, set MAC, re-enable
sudo ip link set dev eth0 down
sudo ip link set dev eth0 address 00:xx:xx:xx:xx:xx
sudo ip link set dev eth0 up

# Verify the new MAC
ip link show eth0
```

---

## 4) Renew/Obtain IP via DHCP

```bash
# Release lease and request a new one
sudo dhclient -r eth0
sudo dhclient -v eth0

# Verify assigned IP and connectivity
ip addr show eth0
ip route
ping -c 3 8.8.8.8
```

---

## 5) Fetch/Test the Lab Page (curl)

Replace `HOSTNAME` with the host you find in the Network tab (e.g., `fakebank.thm` or the displayed host).

```bash
# Verbose test
curl -v http://fakebank.thm

# Extract the flag if it's in the HTML in THM{...} format
curl -s http://fakebank.thm | sed -n 's/.*\(THM{[^}]*}\).*/\1/p'
```

---

## 6) Restore Original MAC (Example with your hardware MAC)

Replace `00:xx:xx:xx:xx:xx` with your original MAC if different.

```bash
sudo ip link set dev eth0 down
sudo ip link set dev eth0 address 08:00:27:ee:79:cc
sudo ip link set dev eth0 up

# Renew DHCP (optional)
sudo dhclient -r eth0
sudo dhclient -v eth0

# Verify
ip link show eth0
ip addr show eth0
ping -c 3 8.8.8.8
```

---

## 7) Set MAC Persistently with NetworkManager (Optional)

If you use NetworkManager and want the MAC to remain set for a specific connection:

```bash
# Show active connections and get the name (e.g., "Wired connection 1")
nmcli connection show --active

# Modify the connection (replace "CONNECTION_NAME" and the desired MAC)
sudo nmcli connection modify "CONNECTION_NAME" 802-3-ethernet.cloned-mac-address 00:12:32:2F:33:39

# Restart the connection to apply
sudo nmcli connection down "CONNECTION_NAME" && sudo nmcli connection up "CONNECTION_NAME"

# To remove (restore) the setting:
sudo nmcli connection modify "CONNECTION_NAME" 802-3-ethernet.cloned-mac-address ""
sudo nmcli connection down "CONNECTION_NAME" && sudo nmcli connection up "CONNECTION_NAME"
```

---

## 8) Make DHCP Automatic at Boot (Optional)

If you prefer eth0 to use DHCP automatically by editing `/etc/network/interfaces`:

```bash
sudo tee /etc/network/interfaces <<'EOF'
auto lo
iface lo inet loopback

auto eth0
iface eth0 inet dhcp
EOF

# Restart networking or the system (or use):
sudo systemctl restart networking
```

---

## 9) Use `macchanger` (Handy, Optional)

If you want to use macchanger (more features):

```bash
sudo apt install -y macchanger

# Show permanent and current MAC
sudo macchanger -s eth0

# Set a specific MAC
sudo ip link set dev eth0 down
sudo macchanger -m 00:12:32:2F:33:39 eth0
sudo ip link set dev eth0 up

# Restore original MAC
sudo macchanger -p eth0
```

---

## 10) Network Debugging (tcpdump) — See traffic while you click "Request Site"

```bash
# Capture http/https packets to the lab host (replace IP or host)
sudo tcpdump -n -i eth0 host 172.66.164.239 and port 443

# Or capture all HTTP/WS to see if traffic is sent:
sudo tcpdump -n -i eth0 port 80 or port 443 -A
# (Ctrl+C to stop)
```

---

## 11) Recommended Cleanup at the End (Restoration)

```bash
# Restore MAC (see above)
sudo ip link set dev eth0 down
sudo ip link set dev eth0 address 00:xx:xx:xx:xx:xx
sudo ip link set dev eth0 up

sudo dhclient -r eth0
sudo dhclient -v eth0
```

---

## Quick Tips and Best Practices

* Write down your **original MAC** (`ip link show eth0`) **first** so you always have it ready.
* Spoof MACs **only in the VM** (not on your host) when doing labs.
* After spoofing, use `dhclient -v eth0` to get an IP.
* If the lab page is accessible via the View Site button, **try copying the MAC shown on the page first** — sometimes it's the quick solution.
* Use `curl` to extract flags quickly (useful if the browser has active protections).
* Always restore the original MAC at the end to avoid network conflicts.